In [1]:
import os
import tensorflow as tf

from heartkit.tasks import TaskFactory
from typing import Type, TypeVar
from argdantic import ArgField, ArgParser
from pydantic import BaseModel
from heartkit.utils import env_flag, set_random_seed, setup_logger

from heartkit.tasks.AFIB_Ident.utils import (
    create_model,
    load_datasets,
    load_test_datasets,
    load_train_datasets,
    prepare,
)

from heartkit.defines import (
    HKDemoParams
)
from heartkit.tasks.AFIB_Ident.defines import (
    get_class_mapping,
    get_class_names,
    get_class_shape,
    get_classes,
    get_feat_shape,
)

cli = ArgParser()
B = TypeVar("B", bound=BaseModel)


def parse_content(cls: Type[B], content: str) -> B:
    """Parse file or raw content into Pydantic model.

    Args:
        cls (B): Pydantic model subclasss
        content (str): File path or raw content

    Returns:
        B: Pydantic model subclass instance
    """
    if os.path.isfile(content):
        with open(content, "r", encoding="utf-8") as f:
            content = f.read()

    return cls.model_validate_json(json_data=content)


config = 'configs/arrhythmia-100class-2.json'
params = parse_content(HKDemoParams, config)


params.seed = set_random_seed(params.seed)
params.data_parallelism = 8

class_names = get_class_names(params.num_classes)
class_map = get_class_mapping(params.num_classes)
input_spec = (
    tf.TensorSpec(shape=get_feat_shape(params.frame_size), dtype=tf.float32),
    tf.TensorSpec(shape=get_class_shape(params.frame_size, params.num_classes), dtype=tf.int32),
)
# since now we are getting one minute for every frame so it should be 400 / 100 * 15 = 60 seconds
datasets = load_datasets(
    ds_path=params.ds_path,
    frame_size=params.frame_size * 15,
    sampling_rate=params.sampling_rate,
    class_map=class_map,
    spec=input_spec,
    datasets=params.datasets,
)


2024-03-18 14:12:49.821684: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-18 14:12:50.743584: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 14:12:52.270330: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 14:12:52.270382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 14:12:52.533808: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## Let's try Patient10000, 240 minutes == 4 hours == 4 * 60 * 60 == 14400 seconds
400 is the model input size and 100HZ is our sampling rate, meaning model input is 4 second, every hour we will have 3600 seconds, thus we should run the prediction 900(3600/4) times for every hour

In [3]:
patient_ids = datasets[0].get_test_patient_ids()
single_pat_gen = datasets[0].uniform_patient_generator(patient_ids=[patient_ids[0]], repeat=False, shuffle=False)
# we should get 4 hours == 4 * 60 * 60
continuous_gen = datasets[0].continous_signal_label_generator(single_pat_gen, 240) # since this is larger than 60, we will have multiple segments together
tmp = next(continuous_gen)
tmp


Starting idx is: 32493, and end idx is: 932493
Starting idx is: 44208, and end idx is: 944208
Starting idx is: 43665, and end idx is: 943665
Starting idx is: 32174, and end idx is: 932174


[{'x': array([-0.07634667, -0.03246027, -0.05736641, ..., -0.08710624,
         -0.02468609, -0.15256278], dtype=float32),
  'frame_rlabel': array([[  1544,      4],
         [ 10934,      1],
         [ 13229,      4],
         [ 16822,      1],
         [ 18360,      4],
         [137627,      1],
         [139796,      4],
         [143350,      1],
         [145577,      4],
         [171319,      1],
         [175930,      4],
         [178796,      1],
         [183632,      4],
         [260563,      1],
         [262595,      4],
         [265225,      1],
         [267227,      4],
         [328470,      1],
         [330218,      4],
         [334495,      1],
         [339517,      4],
         [341128,      1],
         [345264,      4],
         [348399,      1],
         [353861,      4],
         [357077,      1],
         [360231,      4],
         [361314,      1],
         [365203,      4],
         [366328,      1],
         [368728,      4],
         [369872,      1

## Do the prediction and measure the performance

In [8]:
import datetime
import random
import numpy as np
from tqdm import tqdm
from heartkit.rpc.backends import EvbBackend, PcBackend
from IPython.display import clear_output
from heartkit.tasks.AFIB_Ident.utils import (
    create_model,
    load_datasets,
    load_test_datasets,
    load_train_datasets,
    prepare,
)

from enum import IntEnum
from heartkit.defines import (
    HKDemoParams, HeartBeat, HeartRate, HeartRhythm, HeartSegment
)

from heartkit.tasks.AFIB_Ident.defines import (
    get_class_mapping,
    get_class_names,
    get_class_shape,
    get_classes,
    get_feat_shape,
)

class IcentiaRhythm(IntEnum):
    """Icentia rhythm labels"""
    noise = 0
    normal = 1
    afib = 2
    aflut = 3
    end = 4

HeartRhythmMap = {
    IcentiaRhythm.noise: HeartRhythm.noise,
    IcentiaRhythm.normal: HeartRhythm.normal,
    IcentiaRhythm.afib: HeartRhythm.afib,
    IcentiaRhythm.aflut: HeartRhythm.aflut,
    IcentiaRhythm.end: HeartRhythm.noise,
}


BackendRunner = EvbBackend if params.backend == "evb" else PcBackend
runner = BackendRunner(params=params)
whole_seg_pred = []
for i in range(len(tmp)):
    clear_output(wait=True)
    x = tmp[i]['x']
    # y_sig  = signal_label[1]
    y_sig = tmp[i]['frame_rlabel']
    segment_id = tmp[i]['segment_id']
    y_pred = np.zeros(x.shape[0], dtype=np.int32)
    runner.open()
    class_map = get_class_mapping(params.num_classes)
    tgt_labels = list(set(class_map.values()))
    tgt_map = {k: class_map.get(v, -1) for (k, v) in HeartRhythmMap.items()}
    y_sig = y_sig[np.where(~np.isin(y_sig[:, 1], [IcentiaRhythm.noise.value, IcentiaRhythm.end.value]))] # filter the noise and end
    y_orig = np.vectorize(tgt_map.get, otypes=[int])(y_sig[:, 1]) # from 0-4 to 0-3
    print(y_orig)
    if len(y_orig) == 0:
        print("Unidentified label")
        y_orig = np.full(x.shape[0], -1)
    elif len(y_orig) == 1:
        y_orig = np.full(x.shape[0], y_orig[0])
    else: # a more complicated cases where you have AFib mixed with AFlut
        # let's do majority voting here
        print("Majority voting for multi-rlabel case")
        y_orig = np.full(x.shape[0], np.argmax(np.bincount(y_orig)))
        
    tod = datetime.datetime(2024, 5, 24, random.randint(12, 23), 00)
    ts = np.array([tod + datetime.timedelta(seconds=i / params.sampling_rate) for i in range(x.shape[0])])
    # segment_length = 5*params.frame_size
    # print(f"Size of y_orig{len(y_orig)}, and rlabels of {y_orig}")
    # 2000/400 = 5
    row_idx = 0
    ratios = [] # store whether predctions == ground truth for 60 minutes
    for i in tqdm(range(0, x.shape[0], params.frame_size), desc="Inference"):
        if i % (5*params.frame_size) == 0:
            #ts = np.array([tod + datetime.timedelta(seconds=i-row_idx*5*params.frame_size / params.sampling_rate) for i in range(x.shape[0])])
            # start a new row for the make_plots
            row_idx += 1
            # print(row_idx)
        # this is [x.shape[0] - 400, x.shape[0]], get the earlier peak, this is the end
        if i + params.frame_size > x.shape[0]:
            start, stop = x.shape[0] - params.frame_size, x.shape[0]
        else:
            start, stop = i, i + params.frame_size

        # print("Before inference this is the ts:", i, start, stop)
        xx = prepare(x[start:stop], sample_rate=params.sampling_rate, preprocesses=params.preprocesses)
        runner.set_inputs(xx)
        runner.perform_inference()
        yy = runner.get_outputs()
        # y_orig[start:stop] = 
        # this is the predicted label for current frame
        y_pred[start:stop] = np.argmax(yy, axis=-1).flatten()
        # Assuming y_pred and y_orig are numpy arrays
        if y_pred[i] == y_orig[i]:
            ratios.append(1)
        else:
            ratios.append(0)

    print(np.sum(ratios) / (len(y_pred) / 400))
    whole_seg_pred.append(ratios)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
Majority voting for multi-rlabel case


Inference:   0%|          | 0/900 [00:00<?, ?it/s]

1
1/1 [==============================] - 0s 284ms/step


Inference:   0%|          | 1/900 [00:00<04:47,  3.13it/s]

1/1 [==============================] - 0s 23ms/step


...

180
1/1 [==============================] - 0s 20ms/step


Inference: 100%|█████████▉| 898/900 [00:45<00:00, 20.11it/s]

1/1 [==============================] - 0s 21ms/step


Inference: 100%|██████████| 900/900 [00:45<00:00, 19.63it/s]

0.9811111111111112


In [18]:
patient_ids

array([10000, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008,
       10009, 10010, 10011, 10012, 10013, 10014, 10015, 10016, 10017,
       10018, 10019, 10020, 10021, 10022, 10023, 10024, 10025, 10026,
       10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035,
       10036, 10037, 10038, 10039, 10040, 10041, 10042, 10043, 10044,
       10045, 10046, 10047, 10048, 10049, 10050, 10051, 10052, 10053,
       10054, 10055, 10056, 10057, 10058, 10059, 10060, 10061, 10062,
       10063, 10064, 10065, 10066, 10067, 10068, 10069, 10070, 10071,
       10072, 10073, 10074, 10075, 10076, 10077, 10078, 10079, 10080,
       10081, 10082, 10083, 10084, 10085, 10086, 10087, 10088, 10089,
       10090, 10091, 10092, 10093, 10094, 10095, 10096, 10097, 10098,
       10099, 10100, 10101, 10102, 10103, 10104, 10105, 10106, 10107,
       10108, 10109, 10110, 10111, 10112, 10113, 10114, 10115, 10116,
       10117, 10118, 10119, 10120, 10121, 10122, 10123, 10124, 10125,
       10126, 10127,

In [13]:
for i in whole_seg_pred:
    print(np.sum(i) / (len(y_pred) / 400))

0.8944444444444445
0.9966666666666667
0.6266666666666667
0.9811111111111112


In [17]:
for i in range(25, 200):
    i

SyntaxError: invalid character '（' (U+FF08) (1166029156.py, line 1)

In [14]:
whole_seg_pred[2]

[1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [19]:
y_sig = tmp[0]['frame_rlabel']
y_sig

array([[  3686,      4],
       [  5263,      1],
       [ 50787,      4],
       [ 54299,      1],
       [106802,      4],
       [108439,      1],
       [126215,      4],
       [154984,      1],
       [157126,      4],
       [159732,      1],
       [161790,      4],
       [166088,      1],
       [184423,      4],
       [185808,      1],
       [191036,      4],
       [195585,      1],
       [202214,      4],
       [205230,      1],
       [209909,      4],
       [211297,      1],
       [214089,      4],
       [217214,      1],
       [220549,      4],
       [221963,      1],
       [283658,      4],
       [293240,      1],
       [299793,      4],
       [301253,      1],
       [306241,      4],
       [308694,      1],
       [352969,      4],
       [354797,      1]], dtype=int32)

array([[  3686,      4],
       [  5263,      1],
       [ 50787,      4],
       [ 54299,      1],
       [106802,      4],
       [108439,      1],
       [126215,      4],
       [154984,      1],
       [157126,      4],
       [159732,      1],
       [161790,      4],
       [166088,      1],
       [184423,      4],
       [185808,      1],
       [191036,      4],
       [195585,      1],
       [202214,      4],
       [205230,      1],
       [209909,      4],
       [211297,      1],
       [214089,      4],
       [217214,      1],
       [220549,      4],
       [221963,      1],
       [283658,      4],
       [293240,      1],
       [299793,      4],
       [301253,      1],
       [306241,      4],
       [308694,      1],
       [352969,      4],
       [354797,      1]], dtype=int32)

In [22]:
from enum import IntEnum
from heartkit.defines import (
    HKDemoParams, HeartBeat, HeartRate, HeartRhythm, HeartSegment
)


class IcentiaRhythm(IntEnum):
    """Icentia rhythm labels"""
    noise = 0
    normal = 1
    afib = 2
    aflut = 3
    end = 4

HeartRhythmMap = {
    IcentiaRhythm.noise: HeartRhythm.noise,
    IcentiaRhythm.normal: HeartRhythm.normal,
    IcentiaRhythm.afib: HeartRhythm.afib,
    IcentiaRhythm.aflut: HeartRhythm.aflut,
    IcentiaRhythm.end: HeartRhythm.noise,
}


tgt_labels = list(set(class_map.values()))
class_map = get_class_mapping(2)
tgt_map = {k: class_map.get(v, -1) for (k, v) in HeartRhythmMap.items()}
y_sig = y_sig[np.where(~np.isin(y_sig[:, 1], [IcentiaRhythm.noise.value, IcentiaRhythm.end.value]))] # filter the noise and end
y_orig = np.vectorize(tgt_map.get, otypes=[int])(y_sig[:, 1]) # from 0-4 to 0-3
print(y_orig)
if len(y_orig) == 0:
    print("Unidentified label")
    y_orig = np.full(x.shape[0], -1)
elif len(y_orig) == 1:
    y_orig = np.full(x.shape[0], y_orig[0])
else: # a more complicated cases where you have AFib mixed with AFlut
    # let's do majority voting here
    print("Majority voting for multi-rlabel case")
    y_orig = np.full(x.shape[0], np.argmax(np.bincount(y_orig)))


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Majority voting for multi-rlabel case


In [27]:
# Assuming y_pred and y_orig are numpy arrays
ratios = []
for i in range(0, len(y_pred), 400):
    if y_pred[i] == y_orig[i]:
        ratios.append(1)
    else:
        ratios.append(0)

np.sum(ratios) / (len(y_pred) / 400)

0.9861111111111112

In [25]:
np.sum(y_pred == y_orig) / len(y_pred)

0.9861111111111112

In [5]:

# assume all zeros in the beginning
y_pred = np.zeros(x.shape[0], dtype=np.int32)
print(f"Pre-filter QC: {y_sig}")
y_sig = y_sig[np.where(~np.isin(y_sig[:, 1], [IcentiaRhythm.noise.value, IcentiaRhythm.end.value]))] # filter the noise and end
y_orig = np.vectorize(tgt_map.get, otypes=[int])(y_sig[:, 1]) # from 0-4 to 0-3
print(y_orig)
if len(y_orig) == 0:
    print("Unidentified label")
    y_orig = np.full(x.shape[0], -1)
elif len(y_orig) == 1:
    y_orig = np.full(x.shape[0], y_orig[0])
else: # a more complicated cases where you have AFib mixed with AFlut
    # let's do majority voting here
    print("Majority voting for multi-rlabel case")
    y_orig = np.full(x.shape[0], np.argmax(np.bincount(y_orig)))


logger.info("Start generating report on ECG prediction")
# plot the prediction label inside the for loop
fig = make_subplots(
    rows=nrow,
    cols=1,
    specs=[[{"colspan": 1, "type": "xy", "secondary_y": True}]] * nrow,
    subplot_titles=(None, None),
    horizontal_spacing=0.05,
    vertical_spacing=0.1,
)

tod = datetime.datetime(2024, 5, 24, random.randint(12, 23), 00)
ts = np.array([tod + datetime.timedelta(seconds=i / params.sampling_rate) for i in range(x.shape[0])])
# segment_length = 5*params.frame_size
# print(f"Size of y_orig{len(y_orig)}, and rlabels of {y_orig}")
# 2000/400 = 5
for i in tqdm(range(0, x.shape[0], params.frame_size), desc="Inference"):
    if i % (5*params.frame_size) == 0:
        #ts = np.array([tod + datetime.timedelta(seconds=i-row_idx*5*params.frame_size / params.sampling_rate) for i in range(x.shape[0])])
        # start a new row for the make_plots
        row_idx += 1
        print(row_idx)
    # this is [x.shape[0] - 400, x.shape[0]], get the earlier peak, this is the end
    if i + params.frame_size > x.shape[0]:
        start, stop = x.shape[0] - params.frame_size, x.shape[0]
    else:
        start, stop = i, i + params.frame_size
    # print("Before inference this is the ts:", i, start, stop)
    xx = prepare(x[start:stop], sample_rate=params.sampling_rate, preprocesses=params.preprocesses)
    runner.set_inputs(xx)
    runner.perform_inference()
    yy = runner.get_outputs()
    # y_orig[start:stop] = 
    # this is the predicted label for current frame
    y_pred[start:stop] = np.argmax(yy, axis=-1).flatten()
    # print(ts[start], ts[stop-1] - datetime.timedelta(seconds=0.1), y_pred[start])

array([-0.01200302,  0.00307112, -0.00179122, ..., -0.05404   ,
       -0.03833966, -0.04364323], dtype=float32)

## After creating the patient generator, we should extract complete frame from every segment

### 1-minute level

## 15-minute level

## 60-minute level

## 240-minute level

## 1440-minute level

In [19]:
def extract_label_data(segments, tgt_map, tgt_labels, input_size):
    # This maps segment index to segment key
    seg_map: list[str] = list(segments.keys())
    pt_tgt_seg_map = [[] for _ in tgt_labels]
    for seg_idx, seg_key in enumerate(seg_map):
        # Grab rhythm labels
        rlabels = segments[seg_key]["rlabels"][:]

        # Skip if no rhythm labels
        if not rlabels.shape[0]:
            continue
        rlabels = rlabels[np.where(rlabels[:, 1] != IcentiaRhythm.noise.value)[0]]
        # Skip if only noise
        if not rlabels.shape[0]:
            continue

        # Unpack start, end, and label
        xs, xe, xl = rlabels[0::2, 0], rlabels[1::2, 0], rlabels[0::2, 1]

        # Map labels to target labels
        xl = np.vectorize(tgt_map.get, otypes=[int])(xl)

        # Capture segment, start, and end for each target label, we are also grouping them here
        for tgt_idx, tgt_class in enumerate(tgt_labels):
            idxs = np.where((xe - xs >= input_size) & (xl == tgt_class)) # we want to find a large enough frame which does contain the current class of label
            seg_vals = np.vstack((seg_idx * np.ones_like(idxs), xs[idxs], xe[idxs])).T # seg_idx * np.ones_like(idxs) -> create an array filled with the segment index, followed by start and end indices of segment
            pt_tgt_seg_map[tgt_idx] += seg_vals.tolist()
            
    pt_tgt_seg_map = [np.array(b) for b in pt_tgt_seg_map] # pt_tgt_seg_map will be grouped by rlabel
    return pt_tgt_seg_map # all valid data+label per class

In [20]:
import numpy as np

input_size = 400

patient_generator = single_pat_gen
tgt_labels = list(set(class_map.values()))
samples_per_patient = [25, 200]
samples_per_tgt = samples_per_patient
time_duration = [1, 15, 60, 240, 1440] # the key concept here the data we extract is no longer randomly sampled, they are continou for that long duration
selected_time = 15
segment_counter = 0

for _, segments in patient_generator: # going over every patient
        seg_map: list[str] = list(segments.keys()) # every segment, we will have maximal 70 minutes as the ceiling        
        print(f"Grouped sequences per label class {pt_tgt_seg_map}")
        segment_id = np.random.choice(list(segments.keys())) # randomly pick a segment
        print(f"Current segment {cur_seg}")
        segment = segments[segment_id]
        
        # get the overall size of current segment, _sID
        segment_size = segment["data"].shape[0]
        # get all the peak rlabels for current segment
        rlabels = segment["rlabels"][:]

        # xs, xe, xl = rlabels[0::2, 0], rlabels[1::2, 0], rlabels[0::2, 1]
        # randomly select a frame start
        frame_start = np.random.randint(segment_size - input_size * 15 * selected_time)
        frame_end = frame_start + input_size * 15 * selected_time
        # no rlabel
        if len(rlabels) == 0:
            print("Current sample contains no rlables skip it for now")
            continue
        else:
            sig_pos = rlabels[:, 0]
            # print(f"Double check {rlabel}")
            in_range = np.logical_and(frame_start <= sig_pos, frame_end >= sig_pos)
            # Get the indices of the elements that are within the range
            indices = np.where(in_range)[0]
            # current frame does not cover the peak
            if len(indices) == 0:
                # current frame does not contain any peak, get the closet two intervals
                # frame_rlabel = np.array([rlabel[sig_pos <= frame_end][-1], rlabel[frame_start <= sig_pos][0]])
                array1 = rlabels[sig_pos <= frame_end]
                array2 = rlabels[frame_start <= sig_pos]
                # Check if both arrays are non-empty
                if len(array1) > 0 and len(array2) > 0:
                    # If both are non-empty, get the last element of array1 and the first element of array2
                    frame_rlabel = np.array([array1[-1], array2[0]])
                elif len(array1) > 0:
                    # If only array1 is non-empty, get its last element
                    frame_rlabel = np.array([array1[-1]])
                elif len(array2) > 0:
                    # If only array2 is non-empty, get its first element
                    frame_rlabel = np.array([array2[0]])
                else:
                    # If both are empty, set frame_rlabel as an empty array
                    print("Cannot find the sample in current segment")
                    frame_rlabel = np.full(segment_size, 1)
                # if frame_rlabel not in [2,3]:
            else:
                frame_rlabel = rlabels[indices]
            # END if
            frame_rlabel[:, 0] -= frame_start

        x = segment["data"][frame_start:frame_end].squeeze() # this step we are also removing all the position indices for x, x will always be [0, input_size]
        x = np.nan_to_num(x).astype(np.float32)
        if self.sampling_rate != self.target_rate:
            x = pk.signal.resample_signal(x, self.sampling_rate, self.target_rate, axis=0)
        segment_counter += 1
        if segment_counter > selected_time // 60:
            return
    # END FOR
# END FOR